In [1]:
import nltk
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
corpus = api.load('text8')


In [150]:
model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)

In [151]:
import re, string 
import spacy
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)

In [2]:
import json
with open('/Users/silas.rudolf/projects/School/MA/experiments/data/nexoya daily standup 2022-03-17.json', 'r') as f:
    transcript = json.load(f)

speaker_frames = transcript['transcript']['content'][0]['content']
t = []
for x in range(len(speaker_frames)):
    t.append(''.join([i['content'][0]['text'] for i in speaker_frames[x]['content']]))

In [3]:
text = t

In [158]:
nlp = spacy.load('en_core_web_sm') # disabling Named Entity Recognition for speed


In [159]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [160]:
cleaned = [clean_text(i) for i in text]
lemmatized = [lemmatizer(i) for i in cleaned if type(i) == str]

In [7]:
len(''.join(text)) / 20

434.1

In [161]:
all_words = []
for line in lemmatized:
    for word in line.split(' '):
        all_words.append(word)

In [58]:
model.wv.n_similarity('test', 'record')

0.54616284

In [162]:
most_similar = {}

for i in range(len(all_words)-1):
    similarities = []
    for j in range(1, len(all_words)):
        try:
            similarity = model.wv.similarity(all_words[i], all_words[j])
        except:
            continue
        similarities.append(similarity)
    most_similar[all_words[i]] = similarities


In [163]:
to_pop = []
for key,val in most_similar.items():
    if len(val) == 0:
        to_pop.append(key)
for i in to_pop:
    most_similar.pop(i)

import pandas as pd
t = pd.DataFrame(most_similar)

In [164]:
f = t.sum()

In [166]:
f.sort_values(ascending=False)

check      185.733673
get        185.202972
happen     179.735168
look       178.424118
go         174.805237
              ...    
val        -12.962394
clara      -14.532507
philip     -30.676815
ivan       -46.549770
italian    -53.675079
Length: 349, dtype: float32